# Instruct Fine-Tuning Gemma for Persian Language
This notebook demonstrates the fine-tuning of the Gemma model on Persian datasets. We will explore the workflow from data loading and preprocessing to model fine-tuning and evaluation.

**Key Steps:**
1. Setup environment variables for Kaggle and Weights & Biases (wandb).
2. Load and preprocess the Persian Instruct dataset.
3. Set up model parallelism for TPU utilization.
4. Fine-tune the Gemma model using LoRA (Low-Rank Adaptation).
5. Evaluate model performance before and after fine-tuning.


##### you can look into the fine-tuning process logs in here: [link](https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_fa)

#### Device:
we used the TPU VM v3-8 from kaggle.
#### Base model:
we used the fine-tuned version of the gemma2_9b_en which is fine-tuned on Persian(farsi) data, The model [link](https://www.kaggle.com/models/mahdiseddigh/gemma2/keras/gemma2_9b_fa)

# Note:
#### I mistakenly used French data for training a model that I wanted to work with persian, I noticed this after the training was over.
##### How ever the model turned out to be working better on farsi even that it is fine-tuned on french language, I have two possible reasones for this:
- **1- using the fine-tuned base model(on farsi)**
- **2- since the base model was also trained on large amount of multi lingual data, I think the model adapts realtions between the french instruct/response pairs and uses that knowledge in generating for farsi which is a different language**

### My Gemma2 cookbook:
I made this repo and I'm uploading all notebooks related to working with gemma models, check it out:
https://github.com/Mhdaw/Gemma2

### Step 0: Installing the Required Libraries and Frameworks
To ensure that all necessary libraries and frameworks are installed, run the following commands:

In [1]:
!pip install -q -U keras-nlp keras datasets kagglehub keras_hub 
!pip install -q -U tensorflow-text
# Install tensorflow-cpu so tensorflow does not attempt to access the TPU.
!pip install -q -U tensorflow-cpu
!pip install -q -U wandb


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.18.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import jax
jax.devices()

E0000 00:00:1735747919.131944      74 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E0101 16:11:59.171488397     185 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2025-01-01T16:11:59.171471319+00:00"}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

## Step 1: Setup Environment Variables
We will configure the environment variables required for:
- Kaggle API access
- Weights & Biases for tracking experiments
- TensorFlow backend optimization.


In [3]:
import os
# Set the environment variables for Kaggle and Weights & Biases.
# from kaggle_secrets import UserSecretsClient if you use kaggle
# from google.colab import userdata if you use google colab
#import getpass if you use jupyter notebook
os.environ["KAGGLE_USERNAME"] = "your-username"# or UserSecretsClient().get_secret(KAGGLE_USERNAME) or userdata.get(KAGGLE_USERNAME) or getpass.getpass("Enter your KAGGLE_USERNAME: ")
os.environ["KAGGLE_KEY"] = "kaggle-api-key" # or UserSecretsClient().get_secret(KAGGLE_KEY) or userdata.get(KAGGLE_KEY) or getpass.getpass("Enter your  KAGGLE_KEY: ")
os.environ["WANDB_API_KEY"] = "wand-api-key" # or UserSecretsClient().get_secret(WANDB_API_KEY) or userdata.get(WANDB_API_KEY) or getpass.getpass("Enter your WANDB_API_KEY: ")
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [4]:
import tensorflow as tf
import keras
import keras_nlp
from datasets import load_dataset
import itertools
import wandb
from wandb.integration.keras import WandbMetricsLogger

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Load and Explore Persian Dataset
We are using the `CohereForAI/aya_dataset` dataset with Persian insturct data. 

**Subtasks:**
- Load training and validation datasets.
- Extract sample data for exploration.
- Limit dataset size for efficient experimentation.


Since we want to instruct fine-tune the Gemma 2 9b model for adapting to the Persian language, we need a good amount of high-quality Persian instruct and responses. For that, we use the 'Persian' dataset, which is a multilingual instruct dataset.

You can look into it on Hugging Face: [Link](https://huggingface.co/datasets/CohereForAI/aya_dataset)  

**Dataset Summary (from the original dataset page):**  
The Aya Dataset is a multilingual instruction fine-tuning dataset curated by an open-science community via Aya Annotation Platform from Cohere For AI. The dataset contains a total of 204k human-annotated prompt-completion pairs along with the demographics data of the annotators.
This dataset can be used to train, finetune, and evaluate multilingual LLMs.

Curated by: Contributors of Aya Open Science Intiative.

Language(s): 65 languages (71 including dialects & scripts).

License: Apache 2.0

In [5]:
def load_specific_data(target_language="English"):
    aya_dataset = load_dataset("CohereForAI/aya_dataset")
    selected_dataset = aya_dataset.filter(lambda x: x['language'] == target_language)
    return selected_dataset

def generate_text_data(selected_dataset):
    Data = []
    for example in selected_dataset["train"]:
        instruction = example["inputs"]
        response = example["targets"]
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        Data.append(template.format(**{"instruction": instruction, "response": response}))

    test_data = []
    for example in selected_dataset["test"]:
        instruction = example["inputs"]
        response = example["targets"]
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        test_data.append(template.format(**{"instruction": instruction, "response": response}))
    return Data, test_data

In [ ]:
subset = load_specific_data("French")#My mistake!
train_text_data, test_text_data= generate_text_data(subset)

In [7]:
#Since there is no example for our target language in the test subset, we take a small fraction of train data:
train_text_data, test_text_data = train_text_data[:-200], train_text_data[-200:]

In [8]:
# Check the first example to ensure loading is correct
print("First training example:", train_text_data[0],"\n")
print("First validation example:", test_text_data[0])
print(f'\ntraining length:{len(train_text_data)}')

First training example: Instruction:
Quels président des États-Unis ne s’est jamais marié ?

Response:
James Buchanan est le seul président qui ne s'est jamais marié. 

First validation example: Instruction:
Quels sont les 10 termes clés que je devrais connaître sur le changement climatique ? Définissez-les brièvement.

Response:
Bien sûr! Voici 10 termes clés liés au changement climatique accompagnés de brèves définitions :

Changement climatique : altération à long terme des conditions météorologiques moyennes de la Terre, souvent causée par des activités humaines qui libèrent des gaz à effet de serre dans l'atmosphère.

Effet de serre : Processus naturel par lequel certains gaz présents dans l'atmosphère retiennent la chaleur du soleil, réchauffant ainsi la surface de la Terre.

Réchauffement climatique : Augmentation de la température moyenne de la Terre due à l'augmentation de l'effet de serre, principalement causée par les activités humaines.

Dioxyde de carbone (CO2) : Un gaz à 

## Step 3: Data Preprocessing
The text data will be converted into TensorFlow datasets for training and validation. Key preprocessing steps include:
- Creating TensorFlow datasets from plain-text lists.
- Shuffling and batching training data for optimized input.
- Optional text cleaning (if needed).


In [9]:
batch_size = 4

# Convert the lists of text data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices(train_text_data)
val_data = tf.data.Dataset.from_tensor_slices(test_text_data)

# Preprocess each text sample
def preprocess_text(text):
    return tf.convert_to_tensor(text, dtype=tf.string)

# Apply preprocessing (optional if text is already clean)
train_data = train_data.map(preprocess_text)
val_data = val_data.map(preprocess_text)

# Shuffle and batch the training data
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size)
val_data = val_data.batch(batch_size)

## Step 4: Model Parallelism for Efficient Training and Loading the model
We configure model parallelism using TPUs to handle the large-scale Gemma model. Key components:
- **Device Mesh:** A mapping of TPU devices.
- **Layout Map:** Specifies the sharding strategy for different layers.
- Then we load the model in parallel devices.


## Step 5: Model Overview
We initialize the Gemma model for fine-tuning and explore its architecture.

### Key Model Parameters:
- **Model ID:** Pretrained Gemma version for transfer learning.
- **LoRA:** Enable Low-Rank Adaptation for fine-tuning.
- **Sequence Length:** Adjusted for task requirements.


In [10]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)

model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded on 8 TPUs
layout_map["token_embedding/embeddings"] = (model_dim, None)
# Regex to match against the query, key and value matrices in attention layers
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)

model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

keras.distribution.set_distribution(model_parallel)
model_id = "/kaggle/input/gemma2/keras/gemma2_9b_fa/1" # Or /kaggle/input/m/keras/gemma2/keras/gemma2_instruct_2b_en/2
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (34.43 GB)

 Trainable params: 9,241,705,984 (34.43 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
decoder_block_1 = gemma_lm.backbone.get_layer('decoder_block_1')
print(type(decoder_block_1))
for variable in decoder_block_1.weights:
  print(f'{variable.path:<48}  {str(variable.shape):<14}  {str(variable.value.sharding.spec)}')

<class 'keras_hub.src.models.gemma.gemma_decoder_block.GemmaDecoderBlock'>
decoder_block_1/pre_attention_norm/scale          (3584,)         PartitionSpec(None,)
decoder_block_1/post_attention_norm/scale         (3584,)         PartitionSpec(None,)
decoder_block_1/attention/query/kernel            (16, 3584, 256)  PartitionSpec('model', None, None)
decoder_block_1/attention/key/kernel              (8, 3584, 256)  PartitionSpec('model', None, None)
decoder_block_1/attention/value/kernel            (8, 3584, 256)  PartitionSpec('model', None, None)
decoder_block_1/attention/attention_output/kernel  (16, 256, 3584)  PartitionSpec('model', None, None)
decoder_block_1/pre_ffw_norm/scale                (3584,)         PartitionSpec(None,)
decoder_block_1/post_ffw_norm/scale               (3584,)         PartitionSpec(None,)
decoder_block_1/ffw_gating/kernel                 (3584, 14336)   PartitionSpec(None, 'model')
decoder_block_1/ffw_gating_2/kernel               (3584, 14336)   Partition

In [12]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

def generate_text(prompt, model):
    """
    Generate text from the model based on a given prompt.
    """
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    model.compile(sampler=sampler)
    output = model.generate(prompt, max_length=512)
    return output

## Step 6: Evaluate Model Performance Before Fine-Tuning
Before training, test the model on a set of prompts to benchmark its initial performance. This helps us compare improvements after fine-tuning.


In [13]:
# Sample prompt to check performance before and after fine-tuning
test_prompts = [
    "سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن.", # Greeting and request for recent information
    "راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟", # Request for historical knowledge and cultural impact
    "یه شعر کوتاه به فارسی درباره‌ی یه منظره‌ی پاییزی بنویس.", # Request for poetic creativity
    "به زبان ساده توضیح بده که هوش مصنوعی چطور کار می‌کنه و رایج‌ترین کاربردهاش تو ایران چیه.", # Request for technical explanation and geographical context
    "اگه کسی بگه: 'پا رو از گلیمش درازتر کرده'، یعنی چی؟ تو چه موقعیتی می‌شه از این اصطلاح استفاده کرد؟", # Request for interpretation of an idiomatic expression
]

for prompt in test_prompts:
    print(f"\n--- Model Output Before Fine-Tuning for prompt: {prompt} ---")
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output Before Fine-Tuning for prompt: سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن. ---


Instruction:
سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن.

Response:
I have learned that you have to say "khodahafez" when you end a phone call.



--- Model Output Before Fine-Tuning for prompt: راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟ ---


Instruction:
راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟

Response:
رنانسانس به دوران پس از تاریکی میانه ها و قبل از انقلاب صنعتی اطلاق می گردد که در اروپای مرکزی جریان داشت. در طی سالهای 1450 تا 1550 رنسانس در ایتالیا به عنوان یک پدیده فرهنگی شکل گرفت. در طی قرن ها پیش از آنکه رنسانس آغاز گردد، فرهنگ اروپا در دام تاریکی میانه ها گرفتار بود. در طی تاریکی میانه ها، کلیسای کاتولیک زورگویانه عمل می نمود و آزادی اندیشه را سرکوب می نمود. این امر به فروگرفتن علوم و فرهنگ در طول قرون وسطی منجر گردید. با روی کار آمدن ایده رنسانس، به دلیل تلاشهای دانشمندان و فیلسوفان رنسانسی، اندیشه ای مبنی بر شکاکیت در بین مردم رواج یافت. این امر منجر به روی کار آمدن دانشمندانی همچون کپلر و گاالیله گردید که نظریه های علمی کلیسای کاتولیک را زیر سئوال می بردند. رنسانس همچنین منجر به خلق آثار شاعرانه و زیبای هنری در طی این دوران گردید. رنسانس با انقلاب صنعتی که در طی قرن 19 در اروپا آغاز گردید، با مکانیزمهای جدید صنعتی، برداشتهای جدید مردم از جهان را محدود نمود.



--- M

Instruction:
یه شعر کوتاه به فارسی درباره‌ی یه منظره‌ی پاییزی بنویس.

Response:
که بر چمنگاه سبز گیسوانم
بر سر باغ پر چین و شونه‌ی سر
بر دامن جان کهن و خرم من
بر سنگ دل بی سر و ته من
که برگردانم برگ های پاییزی
برگ های زرد و سرخ و نارنجی
که زرد و سرخ و نارنجی اند
بر شاخه های درخت تنومند
تنومند و خشک و شکسته شده
شکسته شده از دست زمان
از دست زمان بزرگ و سفید
برگ های زرد و سرخ و نارنجی
برگ های تو ای عاشق من
عاشق من دیوانه و شیدا
شیدا و بیابان و پر بلا
بیابان و بیواسته شده
و نگاه های پاییزی من
پاییزی ام از نگاه های تو
تو و عشقی که سر به فلک کشیده
سربه فلک و خشک و بی آبی
بی آبی از روی دامن آتش
از دامن آتش تامی
تامیمی در دل زمستانی
در دل زمستانی من امشب
امشب پاییز مرا ببوس
بوس پاییزی و عاشقانه
عاشقانه و بی جان و سرگسسته
سرگسسته از عشق بی جواب
بی جواب و نگاه های پاییزی
نگاه های پاییزی من امشب
پاییز من امشب بگو مرا
بگو زمستان عاشقانه ای
عاشقانه ای در راه است
در راه و برگ های پاییزی
برگ های پاییزی من امشب
پاییز من بگو مرا ببوس
بوس مرا با نگاه های پاییزی
نگاه های پاییزی من امشب
یه آهنگ ترکیبی از 

Instruction:
به زبان ساده توضیح بده که هوش مصنوعی چطور کار می‌کنه و رایج‌ترین کاربردهاش تو ایران چیه.

Response:
درک و برنامه‌ریزی برای انجام کارهایی که معمولاً به درک پیچیده‌ای از جهان و نحوه عمل آن نیاز دارند. کاربردهای آن در بسیاری از زمینه‌ها مانند تشخیص پزشکی، مدیریت دارایی، تجزیه و تحلیل حقوقی و بسیاری موارد دیگر دیده می‌شود. این فناوری همچنین می‌تواند به کسب‌وکارها در بهبود خدمات مشتری، اتوماسیون فرآیندها، پیش‌بینی نتایج و تصمیم‌گیری بهتر و نوآوری و توسعه محصول جدید کمک کند.
Artificial intelligence (AI) is the science of making machines or computer systems work like humans. AI systems can perceive their environment, learn from experience, reason logically, communicate naturally using language, and exercise good judgment. Common applications of artificial intelligence include speech recognition, image recognition, decision-making, natural language processing, and machine learning. AI can also help businesses improve customer service, automate processes, make better predictions an

Instruction:
اگه کسی بگه: 'پا رو از گلیمش درازتر کرده'، یعنی چی؟ تو چه موقعیتی می‌شه از این اصطلاح استفاده کرد؟

Response:
وقتی کلاً همه چی به دست یه نفره و اونم خیلی ساده و بی‌خود، اصلاً به چشم نمیاد، اون موقع میگیم پا رو از گلیمش درازتر کرده.
برای مثال: آقا جون، خواهری دارم که مجرده، میشه با یکی از آشنایان و خوبانتون آشنا بشی؟ - چشماležit، تو کل تهران یه دختر خوب نیافتی؟ - آفرین داداشی، امشب برا کانال منم ویدیو میاری؟
اگه کسی پا رو از گلیمش درازتر کرده، یعنی خیلی کارهای اونو انجام داده، در حالی که اون خیلی کارهای ساده‌ایو انجام داده. گلیمش هم به اندازه‌ای که کارهاشو انجام بده، ولی این شخص خیلی فراتر از اون کاری کرده و خیلی بهتر انجام داده، یعنی پا رو از گلیمش درازتر کرده.
برای مثال: - این اتاق خیلی کوچیکه، بذار فردا یه وانت اینجا خالی کنم و یه اتاق بزرگ بسازم. - نه بابا، خودت یه شب بیا اینجا تمیز کنی و تخت رو بکن، من کارم با این کار با این گلیمته.
یکی از انواع فساد که در این کشور رواج دارد "فقاهت" است.
«فقیه» کسی است که پاى از «گلیم» خود درازتر كند و كارهاى بيش تر از آنچه باید انجام 

## Step 7: Fine-Tuning the Gemma Model with LoRA
We apply LoRA to enable efficient parameter updates during fine-tuning. Key configurations include:
- Optimizer: AdamW with weight decay for transformer models.
- Metrics: Sparse Categorical Accuracy.
- LoRA Rank: Defines the dimensionality of updates.

We use Weights & Biases to monitor training progress and metrics.


In [14]:
LoRA_rank = 8 # you can modify this 
# Enable LoRA for the model and set the LoRA rank to 2,4,...
gemma_lm.backbone.enable_lora(rank=LoRA_rank)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,270,779,392 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,270,779,392 (34.54 GB)

 Trainable params: 29,073,408 (110.91 MB)

 Non-trainable params: 9,241,705,984 (34.43 GB)

In [15]:
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.02,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

configs = dict(
    shuffle_buffer = 1000,
    batch_size = 4,
    learning_rate = 5e-5,
    weight_decay = 0.02,
    sequence_length = 512,
    epochs = 20
)

wandb.init(project = "fine-tuning-gemma2_instruct_9b_fa",
    config=configs
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: this-is-the-way-2005 (this-is-the-way-2005-independent). Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.19.1


wandb: Run data is saved locally in /kaggle/working/wandb/run-20250101_162002-xisx9whp
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run still-glade-1


wandb: ⭐️ View project at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_fa


wandb: 🚀 View run at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_fa/runs/xisx9whp


### Step 8: Training the gemma model:
we train the gemma language model on our ```train_data``` and evaluate it on our ```val_data```, to save time and computation lets use small epochs like 20, If you have more time and computation available, go ahead and increase this!

In [ ]:
# Fit the model
history = gemma_lm.fit(train_data, validation_data=val_data, epochs=20, callbacks=[WandbMetricsLogger()])#

## Step 9: Evaluate Model Performance After Fine-Tuning
Finally, evaluate the fine-tuned model using the same prompts as earlier. Compare the responses to assess improvements in quality and relevance.


In [18]:
test_prompts = [
    "سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن.", # Greeting and request for recent information
    "راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟", # Request for historical knowledge and cultural impact
    "یه شعر کوتاه به فارسی درباره‌ی یه منظره‌ی پاییزی بنویس.", # Request for poetic creativity
    "به زبان ساده توضیح بده که هوش مصنوعی چطور کار می‌کنه و رایج‌ترین کاربردهاش تو ایران چیه.", # Request for technical explanation and geographical context
    "اگه کسی بگه: 'پا رو از گلیمش درازتر کرده'، یعنی چی؟ تو چه موقعیتی می‌شه از این اصطلاح استفاده کرد؟", # Request for interpretation of an idiomatic expression
]

for prompt in test_prompts:
    print(f"\n--- Model Output After Fine-Tuning for prompt: {prompt} ---")
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output After Fine-Tuning for prompt: سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن. ---


Instruction:
سلام! امروز چطوری؟ یه چیز جالب که اخیراً یاد گرفتی رو برام تعریف کن.

Response:
امیدوارم حالت خوب باشه، امروز. 

چیزی که یاد گرفتم اینه که زمین به اندازه کره زمین نیست!
واقعیم وقتی می گی؟

تازه بود؟

تو کلاس یادت کردن اینو؟

خب خب چرا اینو یاد نگیرن؟

من تا دو سه وقتی فکر می کردم زمین دومین جسم بزرگ منظومه ی کهکشونیه، بعد اینář متوجه شدم یه چیزهای ساده تر وجود دارند که بزرگترند از زمین مثل جjupiter،saturne،inch این،اون،و... هست

قبلا سوالم رو این بود که چرا تاریکی ترسناک گرفته میشه ازش لذت میبره؟

حالا فکر کردم و جواب خودم و داده یه چون توی تاریکی میشه چیزهای مخفی乎 دیگه دید








--- Model Output After Fine-Tuning for prompt: راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟ ---


Instruction:
راجع به تاریخ رنسانس در ایتالیا چی می‌دونی؟ می‌تونی تأثیرش رو روی هنر و علم توضیح بدی؟

Response:
Totally استدلال شما بر این اساس است که رنسانس یک نوع غذا است و در عین حال انقلاب فرهنگی غذا بوده است.


--- Model Output After Fine-Tuning for prompt: یه شعر کوتاه به فارسی درباره‌ی یه منظره‌ی پاییزی بنویس. ---

Instruction:
یه شعر کوتاه به فارسی درباره‌ی یه منظره‌ی پاییزی بنویس.

Response:
زمانی که فصل درختان است
عشق
با من آید
نگه دارد بر شاخه ها
و باران را میسپارم به نگاهش

انتهای باز:
جدی میگوید
فقط آبی
حال میخواهد دختر:
دلم را روش






--- Model Output After Fine-Tuning for prompt: به زبان ساده توضیح بده که هوش مصنوعی چطور کار می‌کنه و رایج‌ترین کاربردهاش تو ایران چیه. ---

Instruction:
به زبان ساده توضیح بده که هوش مصنوعی چطور کار می‌کنه و رایج‌ترین کاربردهاش تو ایران چیه.

Response:
حوزه‌ی هوش مصنوعی خیلی گسترده است و به دانش و تکنیک‌های زیادی وابسته است، از فیزیک تا روان‌شناسی.
در حال حاضر، یادگیری عمیق، زیرمجموعه‌ای از ماشین‌های یادگیری، کاربرد زیادی داره. این تکنیک برای بیمارهای سرطان، تحلیل DNA بیماران و ارائه بهترین درمان‌هاشون کمک می‌کنه و در اینستاگرام برای پیدا کردن پست‌های دلخواه شما، تا حدودی جای خود را باز کرده است.





--- Model Output After Fine-Tuning for prompt: اگه کسی بگه: 'پا رو از گلیمش درازتر کرده'، یعنی چی؟ تو چه موقعیتی می‌شه از این اصطلاح استفاده کرد؟ ---


Instruction:
اگه کسی بگه: 'پا رو از گلیمش درازتر کرده'، یعنی چی؟ تو چه موقعیتی می‌شه از این اصطلاح استفاده کرد؟

Response:
خیلی از موقع ها ممکنه این جمله رو بشنویم و باید پاسخی هم داشته باشیم!
'پا رو از گلیمش درازتر کرده' یعنی کسی یه مسئولیتی به عهده گرفته که از توانایی های او خارج هست یا شاید انجام یه کار خیلی بزرگ که دیگران فکر کرده بودن که اون قادر به انجامش نیست ولی اون انجامش کرده و باعث تعجب دیگران شده است.
توی موارد جدی تر شاید یه نفر یه پروژتو به عهده گرفته که فکر نمیره اون تنها با مهارت های معمولی خودش این پروژرو تا تاریخ تحویلش نگه نگه داشته و گمونیم که بقیه این طرز فکر خودشون رو توی سروش بیان می کنن و اون شخص شاید خیلی تحت تاثیر این حرف ها مرده ولی یه ماه بعد یه محصول عالی با تمام مشخصات فرست داده و همه تعجب کردن.
یا توی موارد خیلی ساده و بی اهمیت شاید یه نفر یه مسابقه گذاشته که همه فکر کرده بودن از دست رفته و و و و یه مدت بعد اون برنده فر مشخص می کنه و همه تعجب می کنن.
البته به جای 'تش' می شنه 'تشت' هم مسموعه و خیلی از عزیزانی که این روزها با این سایت آشتی کردن به خاطر این 

#### If you look into our examples and compare it, you can see the models generation has improved for our target language.(even when we are using another language
Note: since this is a fine-tuned model of a base gemma(fine-tuned for faris) model and used instruct and response text in target language, we can expect some randomness and other things from its answers, as it has been fine-tuned on a small instruct datasets and for saving computation we limit the LoRA rank and epochs.

### Step 11: Uploading the fine-tuned model to kaggle:
Here we upload the final fine-tuned model to kaggle models so every one can use it!.
we use /kaggle/tmp to save the model, as the model size is larger than kaggle notebooks output directory size.

In [19]:
tmp_model_dir = "/kaggle/tmp/gemma2_instruct_9b_fa"  # Use /kaggle/tmp
preset_dir = "gemma2_instruct_9b_fa"
os.makedirs(tmp_model_dir, exist_ok=True)
gemma_lm.save_to_preset(tmp_model_dir)

print(f"Model saved to: {tmp_model_dir}")

Model saved to: /kaggle/tmp/gemma2_instruct_9b_fa

In [ ]:
import kagglehub
import keras_hub
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()

model_version = 1
kaggle_username = kagglehub.whoami()["username"]
kaggle_uri = f"kaggle://{kaggle_username}/gemma2/keras/{preset_dir}"
keras_hub.upload_preset(kaggle_uri, tmp_model_dir)
print("Done!")

# Inference
Here we talk about how we can load the fine-tuned model from kaggle and use it:

**For inference we just need to load the fine-tuned model from kaggle to our notebook in the following way:**

for more info check out [here](https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/)

specificly:

A preset is a directory of configs, weights and other file assets used to save and load a pre-trained model. The preset can be passed as one of:
* 1. 
a built-in preset identifier like 'bert_base_e
* 2. '
a Kaggle Models handle like 'kaggle://user/bert/keras/bert_base_
* 3. n'
a Hugging Face handle like 'hf://user/bert_base
* 4. en'
a path to a local preset directory like './bert_base_en'

**Infrence step by step:**
* 1. Load the fine-tuned model from kaggle models
* 2. After the model is succesfuly loaded, You can use it to generate text in the targeted language
* Good luck:)

In [ ]:
final_model_id = "kaggle://mahdiseddigh/gemma2/keras/gemma2_instruct_9b_fa"
finetuned_gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(final_model_id)
finetuned_gemma_lm.summary()

In [ ]:
test_prompt = #define your prompt...
print("\n--- Fine-tuned Models Output ---")
print(generate_text(template.format(instruction=test_prompt, response=""), finetuned_gemma_lm))

# Conclusion
This notebook showcased the complete workflow for fine-tuning the Gemma model for farsi(but we used french) Instruct dataset. We highlighted:
- Dataset preparation
- Model architecture and parallelism
- Fine-tuning with LoRA
- Performance evaluation pre- and post-training